# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#weather data file
weather_data_path = "../WeatherPy/weather_data.csv"

#read the data file
weather_data_df = pd.read_csv(weather_data_path, dtype="object", encoding="utf-8")
weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kargil,34.57,76.1,41.09,52.0,0.0,3.0,IN,1603250704.0
1,havelock,34.88,-76.9,69.01,94.0,1.0,4.61,US,1603250705.0
2,kulachi,31.93,70.46,77.23,17.0,0.0,0.87,PK,1603250705.0
3,gurgan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bathsheba,13.22,-59.52,80.6,83.0,40.0,6.93,BB,1603250705.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps w/ API key
gmaps.configure(api_key=g_key)

#store lat and lng as locations, and Humidity as the weight
locations = weather_data_df[['Lat', 'Lng']].astype(float).dropna()
humidity_w = weather_data_df['Humidity'].astype(float).dropna()

In [4]:
#Create heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_w, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Ideal weather df
ideal_weather_data = weather_data_df.loc[(weather_data_df['Max Temp'] <'80')& 
                                         (weather_data_df['Max Temp'] >'70')&
                                         (weather_data_df['Wind Speed'] < '10')&
                                         (weather_data_df['Cloudiness'] <'50')]
ideal_weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,kulachi,31.93,70.46,77.23,17.0,0.0,0.87,PK,1603250705.0
170,libertador general san martin,-23.81,-64.79,75.0,60.0,100.0,1.01,AR,1603250744.0
242,puerto leguizamo,-0.19,-74.78,73.04,96.0,100.0,1.3,CO,1603250760.0
247,nongstoin,25.52,91.27,73.44,70.0,0.0,1.77,IN,1603250762.0
289,bousso,10.48,16.71,74.25,39.0,0.0,0.63,TD,1603250771.0
353,sao felix do xingu,-6.64,-51.99,74.95,75.0,35.0,0.25,BR,1603250786.0
388,aripuana,-9.17,-60.63,77.97,68.0,16.0,0.4,BR,1603250795.0
426,carauari,-4.88,-66.9,75.09,89.0,34.0,1.66,BR,1603250803.0
520,gat,31.61,34.76,73.0,100.0,12.0,1.99,IL,1603250607.0
561,manicore,-5.81,-61.3,78.37,73.0,4.0,0.67,BR,1603250834.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create new hotel df
hotel_df = ideal_weather_data
hotel_df.insert(0, "Hotel Name", " ")
hotel_df

,Hotel Name,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,,kulachi,31.93,70.46,77.23,17.0,0.0,0.87,PK,1603250705.0
170,,libertador general san martin,-23.81,-64.79,75.0,60.0,100.0,1.01,AR,1603250744.0
242,,puerto leguizamo,-0.19,-74.78,73.04,96.0,100.0,1.3,CO,1603250760.0
247,,nongstoin,25.52,91.27,73.44,70.0,0.0,1.77,IN,1603250762.0
289,,bousso,10.48,16.71,74.25,39.0,0.0,0.63,TD,1603250771.0
353,,sao felix do xingu,-6.64,-51.99,74.95,75.0,35.0,0.25,BR,1603250786.0
388,,aripuana,-9.17,-60.63,77.97,68.0,16.0,0.4,BR,1603250795.0
426,,carauari,-4.88,-66.9,75.09,89.0,34.0,1.66,BR,1603250803.0
520,,gat,31.61,34.76,73.0,100.0,12.0,1.99,IL,1603250607.0
561,,manicore,-5.81,-61.3,78.37,73.0,4.0,0.67,BR,1603250834.0


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    #base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)

    # convert to json
    hotel_name = hotel_name.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


D:\Python\envs\PythonData\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Python\envs\PythonData\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]].astype(float)

In [9]:
# Add marker layer ontop of heat map
figure_layout = {
    'width':'400px',
    'height':'300px',
    'border':'1px solid black',
    'padding':'1px',
    'margin':'0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Display figure

#Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
#Add the layer to the map
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…